```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

1. **[Tables](#tables)**
2. **[Views](#views)**
3. **[Stored Procedures](#stored-procedures)**
4. **[Protos configurations](#protos-configurations)**


# **DEPENDENCIES**
---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
        "src/utils/conversions.ipynb",
        "src/dataflows/nice_hospitals.ipynb",
        "src/dataflows/nice_intensive_care.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RISK_LEVEL_THRESHOLDS]'))
CREATE TABLE [VWSSTAGE].[RISK_LEVEL_THRESHOLDS] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [RISICO NIVEAU]                               VARCHAR(100) NULL,
    [OMSCHRIJVING RISICONIVEAU]                   VARCHAR(100) NULL,
    [7 DAAGS GEMIDDELDE IC OPNAMES]               VARCHAR(100) NULL,
    [7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES]        VARCHAR(100) NULL,
    [GELDIG VANAF]                                VARCHAR(100) NULL,
    [DATE_LAST_INSERTED]                          DATETIME DEFAULT GETDATE()
);
GO

### STATICS

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[RISK_LEVEL_THRESHOLDS]'))
CREATE TABLE [VWSSTATIC].[RISK_LEVEL_THRESHOLDS] (
  [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
  [RISK_LEVEL_1]      VARCHAR(100) NULL,
  [RISK_LEVEL_2]      VARCHAR(100) NULL,
  [RISK_LEVEL_3]      VARCHAR(100) NULL,
  [IC_LOWER]          INT NULL,
  [IC_UPPER]          INT NULL,
  [HOSP_LOWER]        INT NULL,  
  [HOSP_UPPER]        INT NULL,  
  [GELDIG VANAF]    DATE NULL,
  [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()  
);
GO

### DESTINATIONS

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RISK_LEVEL_NL_HISTORY]'))
CREATE TABLE [VWSDEST].[RISK_LEVEL_NL_HISTORY] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
        --MEASURES
        [HOSP_WEEK_START]                    DATETIME NULL                                        
        ,[HOSP_WEEK_END]                      DATETIME NULL                    
        ,[HOSP_7D_AVG]                        DECIMAL(19,3) NULL                    
        ,[HOSP_INSERTION]                     DATETIME NULL                                        
        ,[IC_WEEK_START]                      DATETIME NULL                  
        ,[IC_WEEK_END]                        DATETIME NULL                  
        ,[IC_7D_AVG]                          DECIMAL(19,3) NULL                
        ,[IC_INSERTION]                       DATETIME NULL                 
        --THRESHOLDS
        ,[RISK_LEVEL_1]                     VARCHAR(100) NULL                      
        ,[RISK_LEVEL_2]                     VARCHAR(100) NULL                    
        ,[RISK_LEVEL_3]                     VARCHAR(100) NULL                    
        ,[IC_LOWER]                         INT NULL                        
        ,[IC_UPPER]                         INT NULL                     
        ,[HOSP_LOWER]                       INT NULL                     
        ,[HOSP_UPPER]                       INT NULL                     
        ,[THRESHOLD_VALID_FROM]             DATETIME NULL                           
        --CONCLUSIONS
        ,[RISK_LEVEL_HOSP]                    INT NULL                    
        ,[RISK_LEVEL_IC]                      INT NULL                  
        ,[RISK_LEVEL]                         INT NULL
        ,[VALID_FROM]                         DATETIME NULL               
        ,[VALID]                              INT NULL          
	,[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_RISK_LEVEL_NL_HISTORY]
        ON [VWSDEST].[RISK_LEVEL_NL_HISTORY]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RISK_LEVEL_NL_HISTORY] 
        ON [VWSDEST].[RISK_LEVEL_NL_HISTORY] (
                [DATE_LAST_INSERTED]
        );
GO

# **VIEWS**

---

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER  VIEW [VWSDEST].[V_RISK_LEVEL_NL] AS
WITH BASE_CTE AS
(
    --RETRIEVE ONLY VALID RECORDS
    SELECT 
        [RISK_LEVEL]              
        ,[HOSP_WEEK_START]
        ,[HOSP_WEEK_END]  
        ,[HOSP_7D_AVG]    
        ,[IC_WEEK_START]  
        ,[IC_WEEK_END]    
        ,[IC_7D_AVG]       
        ,[VALID_FROM]                   
        ,[DATE_LAST_INSERTED]               
    FROM [VWSDEST].[RISK_LEVEL_NL_HISTORY] WITH(NOLOCK)
    WHERE [VALID] = 1    
        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RISK_LEVEL_NL_HISTORY] WITH(NOLOCK))
)
select                           
    [risk_level]
    ,[dbo].[convert_datetime_to_unix]([hosp_week_start])                    as hospital_admissions_on_date_of_admission_moving_average_rounded_date_start_unix
    ,[dbo].[convert_datetime_to_unix]([hosp_week_end]  )                    as hospital_admissions_on_date_of_admission_moving_average_rounded_date_end_unix
    ,CAST(ROUND(HOSP_7D_AVG,0) AS INT)                                as hospital_admissions_on_date_of_admission_moving_average_rounded
    ,[dbo].[convert_datetime_to_unix]([ic_week_start])                      as intensive_care_admissions_on_date_of_admission_moving_average_rounded_date_start_unix
    ,[dbo].[convert_datetime_to_unix]([ic_week_end]  )                      as intensive_care_admissions_on_date_of_admission_moving_average_rounded_date_end_unix
    ,CAST(ROUND(IC_7D_AVG,0) AS INT)                                  as intensive_care_admissions_on_date_of_admission_moving_average_rounded
    ,[dbo].convert_datetime_to_unix([valid_from])                           as valid_from_unix
    ,[dbo].[convert_datetime_to_unix](cast([date_last_inserted] as date))   as date_unix
    ,[dbo].[convert_datetime_to_unix]([date_last_inserted])                 as date_of_insertion_unix
FROM BASE_CTE
GO

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_INTENSIVE_CARE_ADMISSIONS] AS
WITH RANK_TABLE AS (
    SELECT
        [DATE_OF_STATISTICS_UNIX] AS DATE_UNIX
        ,[IC_ADMISSION] AS ADMISSIONS_ON_DATE_OF_ADMISSION
        ,IC_ADMISSION_7D_AVG AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE
        ,CAST(ROUND(IC_ADMISSION_7D_AVG,0) AS INTEGER) AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED
        ,IC_ADMISSION_NOTIFICATION AS ADMISSIONS_ON_DATE_OF_REPORTING
        ,DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)  AS DATE_OF_INSERTION_UNIX
        ,ROW_NUMBER() OVER (ORDER BY DATE_OF_STATISTICS DESC) AS RANK_DATE
    FROM VWSDEST.RIVM_IC_OPNAME
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.RIVM_IC_OPNAME)
)
SELECT
    DATE_UNIX
    ,ADMISSIONS_ON_DATE_OF_ADMISSION
    --THIS CUTOFF LOGIC BELOW IS ALSO USED FOR THE RISK LEVEL DETERMINATION IN [DBO].[SP_RISK_LEVEL_NL_HISTORY]
    --LAST VALUES WILL BE TRIMMED DUE TO THESE BEING UNDERREPORTED
    ,IIF(RANK_DATE>3,ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE,NULL) AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE
    ,IIF(RANK_DATE>3,ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED,NULL) AS ADMISSIONS_ON_DATE_OF_ADMISSION_MOVING_AVERAGE_ROUNDED
    ,ADMISSIONS_ON_DATE_OF_REPORTING
    ,DATE_OF_INSERTION_UNIX
FROM RANK_TABLE
GO

# **STORED PROCEDURES**

---

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER   PROCEDURE [dbo].[SP_RISK_LEVEL_THRESHOLDS_STATIC_DATA]
AS
BEGIN

    INSERT INTO [VWSSTATIC].RISK_LEVEL_THRESHOLDS (
         RISK_LEVEL_1  
        ,RISK_LEVEL_2  
        ,RISK_LEVEL_3  
        ,IC_LOWER      
        ,IC_UPPER      
        ,HOSP_LOWER    
        ,HOSP_UPPER    
        ,[GELDIG VANAF]
      )

    SELECT * FROM (
        SELECT    
            -- Risk levels
             LAG([OMSCHRIJVING RISICONIVEAU],1) OVER (ORDER BY [RISICO NIVEAU]  ASC)    AS RISK_LEVEL_1
            ,[OMSCHRIJVING RISICONIVEAU]                                                AS RISK_LEVEL_2
            ,LAG([OMSCHRIJVING RISICONIVEAU],1) OVER (ORDER BY [RISICO NIVEAU]  DESC)   AS RISK_LEVEL_3

            --IC Thresholds
            ,CASE WHEN CHARINDEX('-',[7 DAAGS GEMIDDELDE IC OPNAMES]) > 0 
                    THEN LEFT([7 DAAGS GEMIDDELDE IC OPNAMES], CHARINDEX('-',[7 DAAGS GEMIDDELDE IC OPNAMES])-1)
                ELSE NULL
                END AS IC_LOWER
            ,CASE WHEN CHARINDEX('-',[7 DAAGS GEMIDDELDE IC OPNAMES]) > 0 
                    THEN RIGHT([7 DAAGS GEMIDDELDE IC OPNAMES], LEN([7 DAAGS GEMIDDELDE IC OPNAMES]) - CHARINDEX('-',[7 DAAGS GEMIDDELDE IC OPNAMES]))
                ELSE NULL
                END AS IC_UPPER

            --Hospital Thresholds
            ,CASE WHEN CHARINDEX('-',[7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES]) > 0 
                    THEN LEFT([7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES], CHARINDEX('-',[7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES])-1)
                ELSE NULL
                END AS HOSP_LOWER
            ,CASE WHEN CHARINDEX('-',[7 DAAGS GEMIDDELDE IC OPNAMES]) > 0 
                    THEN RIGHT([7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES], LEN([7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES]) - CHARINDEX('-',[7 DAAGS GEMIDDELDE ZIEKENHUISOPNAMES]))
                ELSE NULL
                END AS HOSP_UPPER

            --Validity
            ,CONVERT(datetime,[GELDIG VANAF],103)     AS [GELDIG VANAF]
            FROM [VWSSTAGE].RISK_LEVEL_THRESHOLDS
            WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTAGE].RISK_LEVEL_THRESHOLDS )
    ) AS T0
        WHERE IC_UPPER IS NOT NULL AND IC_LOWER IS NOT NULL -- Add all logic to middle level row

END
GO


### STATIC &rarr; DESTINATION MAPPING

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER   PROCEDURE [dbo].[SP_RISK_LEVEL_NL_HISTORY]
AS
BEGIN

WITH HOSP_ADM_CTE AS (
    --Get the records for which a 7d average will be shown on the dashboard
    SELECT 
         [DATE_OF_STATISTICS]
        ,[WEEK_START]
        ,ROUND(HOSPITALIZED_7D_AVG_CUTOFF,1) AS HOSP_7D_AVG
        ,DATE_LAST_INSERTED
    FROM [VWSDEST].[NICE_HOSPITAL_NL]
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[NICE_HOSPITAL_NL] )
      AND [HOSPITALIZED_7D_AVG_CUTOFF] IS NOT NULL
)
,
HOSP_ADM_KPI_CTE AS (
    --Get the latest value on which the risk level is based.
    SELECT 
             [DATE_OF_STATISTICS]
            ,[WEEK_START]
            ,[HOSP_7D_AVG] 
            ,DATE_LAST_INSERTED
    FROM HOSP_ADM_CTE
    WHERE [DATE_OF_STATISTICS] = (SELECT MAX([DATE_OF_STATISTICS]) FROM HOSP_ADM_CTE)
)
,
IC_ADM_RANK_TABLE_CTE AS (
    SELECT
     [DATE_OF_STATISTICS]
    ,[WEEK_START]
    ,ROUND(IC_ADMISSION_7D_AVG,1) AS IC_7D_AVG
    ,DATE_LAST_INSERTED
    ,ROW_NUMBER() OVER (ORDER BY DATE_OF_STATISTICS DESC) AS RANK_DATE
    FROM VWSDEST.RIVM_IC_OPNAME
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.RIVM_IC_OPNAME)
)
,
--This cutoff logic below is also used for the risk level determination in [VWSDEST].[V_RIVM_INTENSIVE_CARE_ADMISSIONS]
IC_ADM_CTE AS (
    SELECT
         [DATE_OF_STATISTICS]
        ,[WEEK_START]
        ,IIF(RANK_DATE>3,IC_7D_AVG,NULL) AS IC_7D_AVG
        ,DATE_LAST_INSERTED
    FROM IC_ADM_RANK_TABLE_CTE
    WHERE RANK_DATE>3
),
IC_ADM_KPI_CTE AS (
    --Get the latest value on which the risk level is based.
    SELECT 
             [DATE_OF_STATISTICS]
            ,[WEEK_START]
            ,[IC_7D_AVG] 
            ,DATE_LAST_INSERTED
    FROM IC_ADM_CTE
    WHERE [DATE_OF_STATISTICS] = (SELECT MAX([DATE_OF_STATISTICS]) FROM IC_ADM_CTE)
)
,
RISK_HISTORY_CTE AS (
    --Get the last valid risk level record
    SELECT 
        [RISK_LEVEL]
        ,[VALID_FROM]
        ,[DATE_LAST_INSERTED]
    FROM [VWSDEST].[RISK_LEVEL_NL_HISTORY]
    WHERE VALID = 1 
        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RISK_LEVEL_NL_HISTORY] WHERE VALID = 1 ) )
,
KPI_CTE AS (
    SELECT 
                T0.[WEEK_START]                                         AS HOSP_WEEK_START
                ,T0.[DATE_OF_STATISTICS]                                AS HOSP_WEEK_END  
                ,T0.[HOSP_7D_AVG]                                       AS HOSP_7D_AVG    
                ,T0.DATE_LAST_INSERTED                                  AS HOSP_INSERTION

                ,T1.[WEEK_START]                                        AS IC_WEEK_START
                ,T1.[DATE_OF_STATISTICS]                                AS IC_WEEK_END  
                ,T1.[IC_7D_AVG]                                         AS IC_7D_AVG  
                ,T1.DATE_LAST_INSERTED                                  AS IC_INSERTION

                ,T2.[RISK_LEVEL_1]                                         
                ,T2.[RISK_LEVEL_2]                                         
                ,T2.[RISK_LEVEL_3]                                         
                ,T2.[IC_LOWER]                                                 
                ,T2.[IC_UPPER]                                             
                ,T2.[HOSP_LOWER]                                           
                ,T2.[HOSP_UPPER]                                           
                ,T2.[GELDIG VANAF]                                      AS THRESHOLD_VALID_FROM

                ,T3.RISK_LEVEL                                          AS HISTORIC_RISK_LEVEL   
                ,T3.VALID_FROM                                          AS HISTORIC_RISK_VALID_FROM
    FROM
        -- CTE's below all return 1 line, querying multiple tables gives 1 line as output.
         HOSP_ADM_KPI_CTE   AS T0 -- Hospital admissions KPI data
        ,IC_ADM_KPI_CTE     AS T1 -- IC admissions KPI data
        ,(
            SELECT * FROM [VWSSTATIC].[RISK_LEVEL_THRESHOLDS] 
            WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTATIC].RISK_LEVEL_THRESHOLDS )
         ) AS T2 -- Thresholds to determine levels
        ,RISK_HISTORY_CTE AS T3 -- History to determine valid_from date
)
,
NEW_DATA_CTE AS (
    SELECT * 

        --Add aggregated risk level and validity information

        --Add aggregated risk level: Highest (worst) of both
        ,IIF(RISK_LEVEL_HOSP > RISK_LEVEL_IC, RISK_LEVEL_HOSP, RISK_LEVEL_IC) AS RISK_LEVEL

        --If the risk level does not change, the valid_from date should stay the same
        ,CASE WHEN IIF(RISK_LEVEL_HOSP > RISK_LEVEL_IC, RISK_LEVEL_HOSP, RISK_LEVEL_IC) = HISTORIC_RISK_LEVEL
                THEN HISTORIC_RISK_VALID_FROM
              ELSE CAST(GETDATE() AS DATE) -- Take date of today if risk level is updated
              END AS VALID_FROM
        ,1 AS [VALID] --This column can be set to 0 for erroneous records
    FROM (
        SELECT 

            --Measures
            HOSP_WEEK_START
            ,HOSP_WEEK_END  
            ,HOSP_7D_AVG    
            ,HOSP_INSERTION                     
            ,IC_WEEK_START
            ,IC_WEEK_END  
            ,IC_7D_AVG  
            ,IC_INSERTION

            --Thresholds
            ,[RISK_LEVEL_1]                                         
            ,[RISK_LEVEL_2]                                         
            ,[RISK_LEVEL_3]                                         
            ,[IC_LOWER]                                                 
            ,[IC_UPPER]                                             
            ,[HOSP_LOWER]                                           
            ,[HOSP_UPPER]                                           
            ,[THRESHOLD_VALID_FROM]

            --Historic  
            ,HISTORIC_RISK_LEVEL
            ,HISTORIC_RISK_VALID_FROM

            --Conclusions
            ,CASE WHEN HOSP_7D_AVG >= [HOSP_LOWER] AND  HOSP_7D_AVG <= [HOSP_UPPER] --Risk level 2 thresholds are inclusive
                    THEN 2
                    WHEN HOSP_7D_AVG < [HOSP_LOWER]
                    THEN 1   --Risk level 1 threshold is exclusive
                    WHEN HOSP_7D_AVG > [HOSP_UPPER]
                    THEN 3   --Risk level 3 threshold is exclusive 
                ELSE NULL
                END AS RISK_LEVEL_HOSP
            ,CASE WHEN IC_7D_AVG >= [IC_LOWER] AND  IC_7D_AVG <= [IC_UPPER] --Risk level 2 thresholds are inclusive
                    THEN 2
                    WHEN IC_7D_AVG < [IC_LOWER]
                    THEN 1   --Risk level 1 threshold is exclusive
                    WHEN IC_7D_AVG > [IC_UPPER]
                    THEN 3   --Risk level 3 threshold is exclusive 
                ELSE NULL
                END AS RISK_LEVEL_IC
        FROM KPI_CTE
    ) AS T0
)

INSERT INTO VWSDEST.RISK_LEVEL_NL_HISTORY(

        --Measures
         HOSP_WEEK_START
        ,HOSP_WEEK_END  
        ,HOSP_7D_AVG    
        ,HOSP_INSERTION                     
        ,IC_WEEK_START
        ,IC_WEEK_END  
        ,IC_7D_AVG  
        ,IC_INSERTION

        --Thresholds
        ,[RISK_LEVEL_1]                                         
        ,[RISK_LEVEL_2]                                         
        ,[RISK_LEVEL_3]                                         
        ,[IC_LOWER]                                                 
        ,[IC_UPPER]                                             
        ,[HOSP_LOWER]                                           
        ,[HOSP_UPPER]                                           
        ,[THRESHOLD_VALID_FROM]

        --Conclusions
        ,RISK_LEVEL_HOSP
        ,RISK_LEVEL_IC
        ,RISK_LEVEL
        ,VALID_FROM
        ,VALID
    )

SELECT 

            --Measures
             HOSP_WEEK_START
            ,HOSP_WEEK_END  
            ,HOSP_7D_AVG
            ,HOSP_INSERTION                     
            ,IC_WEEK_START
            ,IC_WEEK_END  
            ,IC_7D_AVG
            ,IC_INSERTION

            --Thresholds
            ,[RISK_LEVEL_1]                                         
            ,[RISK_LEVEL_2]                                         
            ,[RISK_LEVEL_3]                                         
            ,[IC_LOWER]                                                 
            ,[IC_UPPER]                                             
            ,[HOSP_LOWER]                                           
            ,[HOSP_UPPER]                                           
            ,[THRESHOLD_VALID_FROM]
                       
            --Conclusions
            ,RISK_LEVEL_HOSP                                        
            ,RISK_LEVEL_IC                                        
            ,RISK_LEVEL                         
            ,VALID_FROM                                        
            ,VALID                                   

FROM NEW_DATA_CTE


END;
GO


# **PROTOS CONFIGURATIONS**
---

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RISK_LEVEL_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'risk_level',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO